In [1]:
import os
import pandas as pd

In [2]:
dir_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', '互联网招聘网站数据-20190109')
files = ['1545993937965.csv', '1545995178836.csv', '1545994719214.csv', '1545993524166.csv', '1545994117111.csv',
         '1545993734962.csv', '1545995482689.csv', '1545994520923.csv', '1545993316117.csv', '1545996463690.csv',
         '1545995875667.csv', '1545996273574.csv', '1545995679248.csv', '1545993098203.csv', '1545995371123.csv',
         '1545996077068.csv', '1545994918682.csv', '1545994993854.csv', '1545994322721.csv']
files.sort()

In [4]:
for file in files:
    with open(os.path.join(dir_path, file), 'rb') as f, open(os.path.join(os.path.dirname(dir_path), file), 'wb') as f1:
        line = f.readline()
        cnt = line.count(b',')
        while line:
            if line.count(b',') != cnt:
                line = line.strip(b'\n')
                line += f.readline()
                continue
            f1.write(line)
            line = f.readline().replace(b'"', b'')

In [ ]:
# dir_path = os.path.dirname(dir_path)
# files_path = [os.path.join(dir_path, file) for file in files]
# df = pd.read_csv(files_path[0], encoding='gb18030', parse_dates=['page_publish_time'],
#                  usecols=['page_url', 'pos_desc', 'page_publish_time', 'pos_count', 'page_province',
#                           'pos_class', 'pos_industry', 'site_name', 'pos_age', 'pos_name', 'pos_welfare',
#                           'page_city', 'old_industry', 'deliver_count', 'pos_salary', 'com_name',
#                           'pos_education', 'pos_experience', 'read_count'])
#
# for file_path in files_path[1:]:
#     df = df.append(pd.read_csv(file_path,
#                                encoding='gb18030',
#                                parse_dates=['page_publish_time'],
#                                usecols=['page_url', 'pos_desc', 'page_publish_time', 'pos_count', 'page_province',
#                                         'pos_class', 'pos_industry', 'site_name', 'pos_age', 'pos_name', 'pos_welfare',
#                                         'page_city', 'old_industry', 'deliver_count', 'pos_salary', 'com_name',
#                                         'pos_education', 'pos_experience', 'read_count']),
#                    ignore_index=False)
#
# df.shape # (16892402, 19)

上方注释的代码执行完后，本 notebook 占用内存已超过25G，故不推荐直接运行上方代码。

根据我们的观察，发现数据中有很多重复的链接，以及公司名和职位名。我们尝试了不同的去重策略，结果如下：

In [ ]:
# df1 = df.drop_duplicates(subset=['page_url'], keep='last')
# df1.shape  # (13680314, 19)

# df1 = df.drop_duplicates(subset=['pos_name', 'com_name'], keep='last')
# df1.shape  # (2950631, 19)

# df1 = df.drop_duplicates(subset=['pos_name', 'com_name', 'page_city'])
# df1.shape  # (3926646, 19)

我们尝试了三种去重方法：
1. 根据`page_url`属性去重，此时还剩余 13680314 条数据。
1. 根据`pos_name`、`com_name` 去重，此时还剩 2950631 条数据。
1. 根据`pos_name`、`com_name` 和 `page_city` 去重，此时还剩 3926646 条数据。

使用不同的去重策略得到的最终数据集大小不同，这让我们对数据集的内容产生了兴趣，我们通过文本编辑器查看了重复的数据之间的关系，发现重复的记录中，`_id`
属性在所有的数据文件中是唯一的，但我们没有使用该属性。对于重复的数据的其他属性，我们发现，`page_url` 重复度很大，我们推测，当同一家公司发布同一个
岗位时，通过编辑岗位的相关信息，可能并不会改变该岗位的链接属性。且有许多重复数据，仅 `deliver_count` 和 `read_count` 不同。综合考虑下，我们
决定使用第三种去重方法，并保留最后一条数据。

In [5]:
dir_path = os.path.dirname(dir_path)
files_path = [os.path.join(dir_path, file) for file in files]
print(f'read {files_path[0]} start')
df = pd.read_csv(files_path[0], encoding='gb18030', parse_dates=['page_publish_time'],
                 usecols=['page_url', 'pos_desc', 'page_publish_time', 'pos_count', 'page_province',
                          'pos_class', 'pos_industry', 'site_name', 'pos_age', 'pos_name', 'pos_welfare',
                          'page_city', 'old_industry', 'deliver_count', 'pos_salary', 'com_name',
                          'pos_education', 'pos_experience', 'read_count'])
print(f'de-duplicate {files_path[0]} start')
df.drop_duplicates(subset=['pos_name', 'com_name', 'page_city'], keep='last', inplace=True)

for file_path in files_path[1:]:
    print(f'read {file_path} start')
    df = df.append(pd.read_csv(file_path,
                               encoding='gb18030',
                               parse_dates=['page_publish_time'],
                               usecols=['page_url', 'pos_desc', 'page_publish_time', 'pos_count', 'page_province',
                                        'pos_class', 'pos_industry', 'site_name', 'pos_age', 'pos_name', 'pos_welfare',
                                        'page_city', 'old_industry', 'deliver_count', 'pos_salary', 'com_name',
                                        'pos_education', 'pos_experience', 'read_count']),
                   ignore_index=False)
    print(f'de-duplicate {file_path} start')
    df.drop_duplicates(subset=['pos_name', 'com_name', 'page_city'], keep='last', inplace=True)

df.shape # (3926646, 19)

read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993098203.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993098203.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993316117.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993316117.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993524166.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993524166.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993734962.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993734962.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993937965.csv start


c:\users\rxg\desktop\code\dataminingbigproject\.venv\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (4,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545993937965.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994117111.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994117111.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994322721.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994322721.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994520923.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994520923.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994719214.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994719214.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994918682.csv start
de-duplicate C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545994918682.csv start
read C:\Users\rxg\Desktop\Code\DataMiningBigProject\data\1545

(3926646, 19)

In [20]:
# save file
df.reset_index(inplace=True, drop=True)
df.to_csv(os.path.join(dir_path, '1-de-duplicate.csv'), index=False)

In [21]:
df.head()

,page_url,pos_desc,page_publish_time,pos_count,page_province,pos_class,pos_industry,site_name,pos_age,pos_name,pos_welfare,page_city,old_industry,deliver_count,pos_salary,com_name,pos_education,pos_experience,read_count
0,http://jobs.zhaopin.com/CC690972424J0013190060...,任职资格： 1.男性，年龄18-40岁；能适应倒班制；要求熟练工； 2.工作地点：江苏徐州丰...,2018-07-17,10.0,410000.0,生产/制造,H018,智联招聘,0.0,数控车床/磨床工,NaN,410300.0,大型设备/机电设备/重工业,0.0,4001-6000,江苏巨杰机电有限公司,中技,1-3年,0.0
1,http://zz.58.com/tech/34770954823372x.shtml?ps...,兼职寒暑假短期工勿扰，不符合条件者勿扰 任职要求： 1，男女不限，18-27岁，专科及以...,2018-07-17,3.0,410000.0,网络/通信/电子,H004,58同城,0.0,初级网站架构设计师餐补,五险一金 周末双休 年底双薪 饭补 加班补助,410100.0,计算机硬件,0.0,4000-6000,河南洽普网络科技有限公司,大专,经验不限，可接收应届生,2.0
2,http://zz.58.com/zhuanye/34568001532484x.shtml...,岗位职责： 1、咨询电话的接听，做好相应的信息记录； 2、定期对记录的咨询信息进行回访，...,2018-07-17,2.0,410000.0,法律/教育/翻译/出版,H007,58同城,0.0,包食宿急聘课程顾问,加班补助 房补 包住 包吃,410100.0,教育/科研/培训,3.0,3000-5000,郑州市二七区你好外语培训学校,大专,1-2年，可接收应届生,102.0
3,http://jobs.zhaopin.com/624002185250004.htm,岗位职责： 1、负责产品的验收评估，并编写产品评估报告； 2、负责产品应用技术研发； 3、解...,2018-07-17,1.0,340000.0,生产/制造,H009,智联招聘,0.0,技术应用工程师,五险一金免费班车,340400.0,石油/石化/化工,0.0,3000-5000?,清远升华新材料科技有限公司,不限,不限,0.0
4,http://cq.58.com/yewu/25035685221419x.shtml?ps...,岗位职责： 1、负责根据客户的要求，给用户提供专业的知识咨询和服务； 2、负责推荐保险种...,2018-07-17,3.0,500000.0,销售/客服/采购/淘宝,H008,58同城,0.0,急急急+销售客户经理,周末双休 免费培训,500100.0,保险,34.0,5000-8000,中国平安人寿保险股份有限公司重庆分公司（平安保险）,学历不限,经验不限，可接收应届生,2153.0


去重完成后，总共 3926646 行，19列数据。

In [24]:
df.count()

page_url             3926646
pos_desc             3926646
page_publish_time    3926646
pos_count            3926646
page_province        3926586
pos_class            3926646
pos_industry         3926646
site_name            3926646
pos_age              3926646
pos_name             3926646
pos_welfare          3637882
page_city            3914544
old_industry         3926646
deliver_count        3926646
pos_salary           3926646
com_name             3926646
pos_education        3926646
pos_experience       3926646
read_count           3926646
dtype: int64